In [1]:
import sys

sys.path.insert(0, "../")

import pandas as pd
from mdu.eval.table_analysis_utils import (
    transform_by_tasks,
    select_composite_and_components,
    check_composite_dominance,
    compute_average_ranks,
    analyze_composite_pareto_performance,
)

# Set pandas display options to show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [2]:
df = pd.read_csv(
    "../per_comp_scaled_benchmark_entropic_target_exp_eps_0.5_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_1.csv"
)

In [3]:
df.head(10)

,ind_dataset,ood_dataset,measure,uncertainty_type,gname,risk_type,gt_approximation,pred_approximation,ensemble_group,problem_type,roc_auc,average_precision,accuracy,aurc,acc_cov_auc,coverage_at_1pct_error,coverage_at_2pct_error,coverage_at_5pct_error,n_ind_samples,n_ood_samples,n_correct,n_incorrect,ensemble_accuracy
0,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,ood_detection,0.912015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963750
1,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,misclassification_detection,0.944999,0.381728,0.963750,NaN,NaN,NaN,NaN,NaN,7200,NaN,6939.0,261.0,0.963750
2,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,selective_prediction,NaN,NaN,0.963750,0.002928,0.996933,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963750
3,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,ood_detection,0.910690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964306
4,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,misclassification_detection,0.944610,0.374033,0.964306,NaN,NaN,NaN,NaN,NaN,7200,NaN,6943.0,257.0,0.964306
5,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,selective_prediction,NaN,NaN,0.964306,0.002930,0.996931,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.964306
6,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,ood_detection,0.911888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963333
7,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,misclassification_detection,0.943641,0.360288,0.963333,NaN,NaN,NaN,NaN,NaN,7200,NaN,6936.0,264.0,0.963333
8,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,selective_prediction,NaN,NaN,0.963333,0.003023,0.996838,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963333
9,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,3,ood_detection,0.911436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964167


In [4]:
df.ood_dataset.unique()

array(['cifar100', 'cifar10', 'tiny_imagenet', 'svhn', 'imagenet_a',
       'imagenet_o', 'imagenet_r'], dtype=object)

In [5]:
transformed_df = transform_by_tasks(df)

In [6]:
transformed_df

measure_pretty                           R_b 1 (Brier)  R_b 1 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.942466          0.942267   
              cifar10 [selective]             0.996836          0.996818   
              cifar100 [ood]                  0.914436          0.916906   
              svhn [ood]                      0.958357          0.962992   
              tiny_imagenet [ood]             0.908611          0.911364   
cifar100      cifar10 [ood]                   0.773000          0.773270   
              cifar100 [miscls]               0.857773          0.845094   
              cifar100 [selective]            0.920203          0.915943   
              svhn [ood]                      0.858293          0.870120   
              tiny_imagenet [ood]             0.789521          0.809888   
tiny_imagenet tiny_imagenet [miscls]          0.854657          0.844739   
              tiny_imagenet [selective]       0.892676          0.888853   
              imagenet_a [ood]                0.827188          0.835350   
              imagenet_r [ood]                0.816241          0.825339   
              imagenet_o [ood]                0.720970          0.724312   

measure_pretty                           R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.942228          0.941776   
              cifar10 [selective]                 0.996827          0.996810   
              cifar100 [ood]                      0.914506          0.913191   
              svhn [ood]                          0.958530          0.956294   
              tiny_imagenet [ood]                 0.908679          0.907210   
cifar100      cifar10 [ood]                       0.773388          0.772177   
              cifar100 [miscls]                   0.856181          0.858986   
              cifar100 [selective]                0.919719          0.920576   
              svhn [ood]                          0.860970          0.855937   
              tiny_imagenet [ood]                 0.806267          0.803022   
tiny_imagenet tiny_imagenet [miscls]              0.853150          0.854657   
              tiny_imagenet [selective]           0.892115          0.892648   
              imagenet_a [ood]                    0.830074          0.826338   
              imagenet_r [ood]                    0.819225          0.815484   
              imagenet_o [ood]                    0.722622          0.721199   

measure_pretty                           R_b 2 (Brier)  R_b 2 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.945782          0.944886   
              cifar10 [selective]             0.996959          0.996920   
              cifar100 [ood]                  0.913049          0.916237   
              svhn [ood]                      0.957686          0.963150   
              tiny_imagenet [ood]             0.905817          0.909549   
cifar100      cifar10 [ood]                   0.774385          0.774538   
              cifar100 [miscls]               0.865555          0.850089   
              cifar100 [selective]            0.922462          0.917541   
              svhn [ood]                      0.856738          0.870178   
              tiny_imagenet [ood]             0.937085          0.926019   
tiny_imagenet tiny_imagenet [miscls]          0.863376          0.849698   
              tiny_imagenet [selective]       0.895661          0.890666   
              imagenet_a [ood]                0.834463          0.840955   
              imagenet_r [ood]                0.824133          0.830950   
              imagenet_o [ood]                0.735803          0.734891   

measure_pretty                           R_b 2 (Spher

In [7]:
transformed_df.shape

(15, 95)

In [8]:
from configs.interesting_compositions import INTERESTING_COMPOSITIONS

for k in INTERESTING_COMPOSITIONS.keys():
    print(k)

COMPOSITE BAYES ALL OUTER
COMPOSITE BAYES ALL INNER
COMPOSITE BAYES ALL CENTRAL
COMPOSITE BAYES ALL OUTER + M
COMPOSITE BAYES ALL INNER + M
COMPOSITE BAYES ALL CENTRAL + M
COMPOSITE EXCESS ALL OUTER OUTER
COMPOSITE EXCESS ALL OUTER OUTER + M
COMPOSITE EAT LOGSCORE OUTER OUTER + M


In [9]:
res_df = select_composite_and_components(transformed_df, "COMPOSITE BAYES ALL OUTER")

res_df

measure_pretty                           R_b 1 (Logscore)  R_b 1 (Brier)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                   0.942267       0.942466   
              cifar10 [selective]                0.996818       0.996836   
              cifar100 [ood]                     0.916906       0.914436   
              svhn [ood]                         0.962992       0.958357   
              tiny_imagenet [ood]                0.911364       0.908611   
cifar100      cifar10 [ood]                      0.773270       0.773000   
              cifar100 [miscls]                  0.845094       0.857773   
              cifar100 [selective]               0.915943       0.920203   
              svhn [ood]                         0.870120       0.858293   
              tiny_imagenet [ood]                0.809888       0.789521   
tiny_imagenet tiny_imagenet [miscls]             0.844739       0.854657   
              tiny_imagenet [selective]          0.888853       0.892676   
              imagenet_a [ood]                   0.835350       0.827188   
              imagenet_r [ood]                   0.825339       0.816241   
              imagenet_o [ood]                   0.724312       0.720970   

measure_pretty                           R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.942228          0.941776   
              cifar10 [selective]                 0.996827          0.996810   
              cifar100 [ood]                      0.914506          0.913191   
              svhn [ood]                          0.958530          0.956294   
              tiny_imagenet [ood]                 0.908679          0.907210   
cifar100      cifar10 [ood]                       0.773388          0.772177   
              cifar100 [miscls]                   0.856181          0.858986   
              cifar100 [selective]                0.919719          0.920576   
              svhn [ood]                          0.860970          0.855937   
              tiny_imagenet [ood]                 0.806267          0.803022   
tiny_imagenet tiny_imagenet [miscls]              0.853150          0.854657   
              tiny_imagenet [selective]           0.892115          0.892648   
              imagenet_a [ood]                    0.830074          0.826338   
              imagenet_r [ood]                    0.819225          0.815484   
              imagenet_o [ood]                    0.722622          0.721199   

measure_pretty                           composite bayes all outer  
ind_dataset   eval                                                  
cifar10       cifar10 [miscls]                            0.942231  
              cifar10 [selective]                         0.996822  
              cifar100 [ood]                              0.914837  
              svhn [ood]                                  0.959316  
              tiny_imagenet [ood]                         0.909076  
cifar100      cifar10 [ood]                               0.773212  
              cifar100 [miscls]                           0.855261  
              cifar100 [selective]                        0.919373  
              svhn [ood]                                  0.861933  
              tiny_imagenet [ood]                         0.801724  
tiny_imagenet tiny_imagenet [miscls]                      0.852620  
              tiny_imagenet [selective]                   0.891891  
              imagenet_a [ood]                            0.830386  
              imagenet_r [ood]                            0.819638  
              imagenet_o [ood]                            0.722528

In [10]:
# Apply the dominance check
res_df_with_dominance = check_composite_dominance(res_df)
res_df_with_dominance

measure_pretty                           R_b 1 (Logscore)  R_b 1 (Brier)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                   0.942267       0.942466   
              cifar10 [selective]                0.996818       0.996836   
              cifar100 [ood]                     0.916906       0.914436   
              svhn [ood]                         0.962992       0.958357   
              tiny_imagenet [ood]                0.911364       0.908611   
cifar100      cifar10 [ood]                      0.773270       0.773000   
              cifar100 [miscls]                  0.845094       0.857773   
              cifar100 [selective]               0.915943       0.920203   
              svhn [ood]                         0.870120       0.858293   
              tiny_imagenet [ood]                0.809888       0.789521   
tiny_imagenet tiny_imagenet [miscls]             0.844739       0.854657   
              tiny_imagenet [selective]          0.888853       0.892676   
              imagenet_a [ood]                   0.835350       0.827188   
              imagenet_r [ood]                   0.825339       0.816241   
              imagenet_o [ood]                   0.724312       0.720970   

measure_pretty                           R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.942228          0.941776   
              cifar10 [selective]                 0.996827          0.996810   
              cifar100 [ood]                      0.914506          0.913191   
              svhn [ood]                          0.958530          0.956294   
              tiny_imagenet [ood]                 0.908679          0.907210   
cifar100      cifar10 [ood]                       0.773388          0.772177   
              cifar100 [miscls]                   0.856181          0.858986   
              cifar100 [selective]                0.919719          0.920576   
              svhn [ood]                          0.860970          0.855937   
              tiny_imagenet [ood]                 0.806267          0.803022   
tiny_imagenet tiny_imagenet [miscls]              0.853150          0.854657   
              tiny_imagenet [selective]           0.892115          0.892648   
              imagenet_a [ood]                    0.830074          0.826338   
              imagenet_r [ood]                    0.819225          0.815484   
              imagenet_o [ood]                    0.722622          0.721199   

measure_pretty                           composite bayes all outer  \
ind_dataset   eval                                                   
cifar10       cifar10 [miscls]                            0.942231   
              cifar10 [selective]                         0.996822   
              cifar100 [ood]                              0.914837   
              svhn [ood]                                  0.959316   
              tiny_imagenet [ood]                         0.909076   
cifar100      cifar10 [ood]                               0.773212   
              cifar100 [miscls]                           0.855261   
              cifar100 [selective]                        0.919373   
              svhn [ood]                                  0.861933   
              tiny_imagenet [ood]                         0.801724   
tiny_imagenet tiny_imagenet [miscls]                      0.852620   
              tiny_imagenet [selective]                   0.891891   
              imagenet_a [ood]                            0.830386   
              imagenet_r [ood]                            0.819638   
              imagenet_o [ood]                            0.722528   

measure_pretty                           if_dominates_100%  if_dominates_75%  \
ind_dataset   eval                                                         

In [11]:
# Compute and display average ranks
avg_ranks = compute_average_ranks(transformed_df)

In [12]:
for name, el in avg_ranks.items():
    print(name, round(el, 3))

R_t 2 3 (Spherical) 15.0
R_t 1 3 (Spherical) 15.067
composite bayes all inner 15.533
R_b 2 (Spherical) 16.2
R_t 2 1 (Spherical) 16.2
R_t 2 2 (Spherical) 16.2
R_b 3 (Brier) 16.267
R_b 2 (Brier) 16.267
R_t 2 3 (Brier) 16.267
R_t 3 3 (Brier) 16.267
R_t 2 2 (Brier) 16.267
R_t 3 2 (Brier) 16.267
R_t 3 1 (Logscore) 16.467
R_t 1 3 (Brier) 16.6
R_t 1 2 (Brier) 16.6
R_t 1 1 (Spherical) 16.8
R_t 3 2 (Logscore) 16.8
R_t 1 2 (Spherical) 17.2
R_t 2 1 (Logscore) 17.6
R_t 1 1 (Logscore) 18.067
R_t 1 2 (Logscore) 18.2
R_b 3 (Logscore) 18.333
R_t 3 3 (Logscore) 18.333
R_t 2 2 (Logscore) 18.333
R_b 2 (Logscore) 18.333
R_t 2 2 (Zero-one) 18.6
R_t 2 3 (Zero-one) 18.6
R_t 3 3 (Zero-one) 18.6
R_b 2 (Zero-one) 18.6
R_t 3 2 (Zero-one) 18.6
R_b 3 (Zero-one) 18.6
R_t 1 3 (Zero-one) 19.067
R_t 1 2 (Zero-one) 19.067
composite bayes all central 19.467
R_t 2 3 (Logscore) 19.667
R_t 1 3 (Logscore) 19.733
R_t 3 1 (Zero-one) 20.0
R_t 2 1 (Zero-one) 20.0
R_t 1 1 (Zero-one) 20.467
composite eat logscore outer outer + m 

In [13]:
composite_pareto_results = analyze_composite_pareto_performance(
    transformed_df, INTERESTING_COMPOSITIONS
)

In [14]:
composite_pareto_results

{'COMPOSITE BAYES ALL OUTER': {'pareto_count': 30,
  'total_problems': 105,
  'pareto_percentage': 28.57142857142857},
 'COMPOSITE BAYES ALL INNER': {'pareto_count': 57,
  'total_problems': 105,
  'pareto_percentage': 54.285714285714285},
 'COMPOSITE BAYES ALL CENTRAL': {'pareto_count': 41,
  'total_problems': 105,
  'pareto_percentage': 39.04761904761905},
 'COMPOSITE BAYES ALL OUTER + M': {'pareto_count': 50,
  'total_problems': 105,
  'pareto_percentage': 47.61904761904761},
 'COMPOSITE BAYES ALL INNER + M': {'pareto_count': 54,
  'total_problems': 105,
  'pareto_percentage': 51.42857142857142},
 'COMPOSITE BAYES ALL CENTRAL + M': {'pareto_count': 36,
  'total_problems': 105,
  'pareto_percentage': 34.285714285714285},
 'COMPOSITE EXCESS ALL OUTER OUTER': {'pareto_count': 6,
  'total_problems': 105,
  'pareto_percentage': 5.714285714285714},
 'COMPOSITE EXCESS ALL OUTER OUTER + M': {'pareto_count': 27,
  'total_problems': 105,
  'pareto_percentage': 25.71428571428571},
 'COMPOSITE E

In [15]:
# Display results summary
if composite_pareto_results:
    print("\n" + "=" * 80)
    print("COMPOSITE PARETO FRONT ANALYSIS SUMMARY")
    print("=" * 80)

    # Sort by pareto percentage
    sorted_results = sorted(
        composite_pareto_results.items(),
        key=lambda x: x[1]["pareto_percentage"],
        reverse=True,
    )

    for composite_name, results in sorted_results:
        print(
            f"{composite_name:40} {results['pareto_count']:3d}/{results['total_problems']:3d} ({results['pareto_percentage']:5.1f}%)"
        )


COMPOSITE PARETO FRONT ANALYSIS SUMMARY
COMPOSITE BAYES ALL INNER                 57/105 ( 54.3%)
COMPOSITE EAT LOGSCORE OUTER OUTER + M    56/105 ( 53.3%)
COMPOSITE BAYES ALL INNER + M             54/105 ( 51.4%)
COMPOSITE BAYES ALL OUTER + M             50/105 ( 47.6%)
COMPOSITE BAYES ALL CENTRAL               41/105 ( 39.0%)
COMPOSITE BAYES ALL CENTRAL + M           36/105 ( 34.3%)
COMPOSITE BAYES ALL OUTER                 30/105 ( 28.6%)
COMPOSITE EXCESS ALL OUTER OUTER + M      27/105 ( 25.7%)
COMPOSITE EXCESS ALL OUTER OUTER           6/105 (  5.7%)
